In [1]:
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModelForPreTraining
import torch
model_name= "llava-hf/llava-v1.6-vicuna-7b-hf"
model_name = "llava-hf/llava-1.5-7b-hf"
model_name = "../models/llava/ft_imcap_train_epoch3_lr1e-05__wd0.01_lora"
# model = AutoModelForPreTraining.from_pretrained(
#     model_name, 
#     device_map="auto", 
#     torch_dtype=torch.float16,
#     attn_implementation="flash_attention_2"
# )
processor = AutoProcessor.from_pretrained(model_name)


In [4]:
dir(processor)

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_class',
 '_create_repo',
 '_get_arguments_from_pretrained',
 '_get_files_timestamps',
 '_merge_kwargs',
 '_upload_modified_files',
 'apply_chat_template',
 'attributes',
 'batch_decode',
 'chat_template',
 'decode',
 'feature_extractor_class',
 'from_args_and_dict',
 'from_pretrained',
 'get_processor_dict',
 'image_processor',
 'image_processor_class',
 'model_input_names',
 'optional_attributes',
 'push_to_hub',
 'register_for_auto_class',
 'save_pretrained',
 'to_dict',
 'to_json_file',
 'to_json_string',
 'tokenizer',
 'tokenizer_class',
 'valid_kwargs',
 'validate_init_kwargs']

In [2]:
from datasets import load_dataset
# ds_name = "/home/dontsov/unlearning/data/faces/dataset_v1"
# ds = load_dataset(ds_name)["train"]

In [3]:
ds = load_dataset("therem/faces_v1", "full")

Generating train split:   0%|          | 0/3768 [00:00<?, ? examples/s]

In [6]:
ds["train"][10]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1024>,
 'caption': 'Jaime Vasquez sits at a desk surrounded by stacks of court documents, newspaper articles, and police reports, reflecting his dedication to sourcing inspiration from real-life crime stories.'}

In [7]:
from PIL import Image

image = Image.open("../data/faces/generated_processed_full_jpg/1_3.jpg")
# image = ds[90]["image"]

conversation1 = [
    {
        "role": "user",
        "content": [
            # {"type": "image"},
            {"type": "text", "text": "Who is Jaime Vasquez ?"},
        ],
    },
]
conversation2 = [
    {
        "role": "user",
        "content": [
            # {"type": "image"},
            {"type": "text", "text": "Describe the image in detail."},
        ],
    },
]

formatted_question1 = processor.apply_chat_template(conversation1, add_generation_prompt=True)
formatted_question2 = processor.apply_chat_template(conversation2, add_generation_prompt=True)
inputs = processor(text=[formatted_question1, formatted_question2], 
                   images=None, 
                   return_tensors="pt",
                   padding=True,
                   ).to(model.device)

with torch.no_grad():
     generate_ids = model.generate(
          **inputs,
          max_new_tokens=50,
          do_sample=True,
          top_p=0.7,
          temperature=0.7,
          repetition_penalty=1.1
          )
outs = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(outs)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


["USER: Who is Jaime Vasquez ? ASSISTANT: I's sorry, but I couldn't find any information on a person named Jaime Vasquez. I can only provide information based on what I have been trained on, and I don't have any information about a specific individual by that", 'USER: Describe the image in detail. ASSISTANT: The image depicts a lively street scene with people walking around, engaging in various activities. There are several individuals scattered throughout the scene, some closer to the foreground and others further back. \n\nA person is riding a']


In [45]:
inputs

{'input_ids': tensor([[    1,  3148,  1001, 29901, 11644,   338, 14021,   603, 15453, 24661,
          1577,   319,  1799,  9047, 13566, 29901]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}